In [ ]:
# Here is the Python code for the robustness tests we discussed.

# The code uses the `linearmodels` library, which is the standard for panel data econometrics in Python (similar to `plm` in R or `xtreg` in Stata). You can install it with `pip install linearmodels`.

# The code first creates a sample dataset so you can run all the examples directly.


import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# --- 1. SETUP: CREATE SAMPLE DATA ---
# This creates a sample DataFrame in the "panel" format you need.
# Your real data should be structured like this.

# Create a date range
dates = pd.date_range(start='2018-01-01', end='2018-12-31', freq='D')
neighborhoods = ['Midtown', 'FiDi', 'DUMBO', 'Upper West Side', 'Chelsea']

# Create a MultiIndex (neighborhood, date)
index = pd.MultiIndex.from_product([neighborhoods, dates], names=['neighborhood', 'date'])
df = pd.DataFrame(index=index).reset_index()

# Create variables
np.random.seed(42)
df['day_of_week'] = df['date'].dt.day_name()
df['day_mean_temp'] = np.random.uniform(30, 85, size=len(df))
df['precipitation'] = np.random.rand(len(df)) * 0.5
df['wind_speed'] = np.random.uniform(5, 20, size=len(df))
df['holiday'] = (df['date'].dt.month.isin([1, 7, 12]) & df['date'].dt.day.isin([1, 4, 25])).astype(int)

# Create the outcome variable (Y)
# Let's assume a base trip of 15 mins + temp effect + neighborhood effect + noise
neighborhood_effect = df['neighborhood'].map({
    'Midtown': 2, 'FiDi': -3, 'DUMBO': 5, 'Upper West Side': 4, 'Chelsea': 1
})
df['avg_trip_minutes'] = (
    15 + 
    0.3 * df['day_mean_temp'] +  # True effect of temp
    neighborhood_effect +
    (df['day_of_week'] == 'Saturday') * 5 + # Weekend effect
    np.random.randn(len(df)) * 2 # Noise
)

# Set the MultiIndex - This is required for PanelOLS
df = df.set_index(['neighborhood', 'date'])

print("--- Sample Data Head ---")
print(df.head())
print("\n")


# --- 2. THE BASELINE TWO-WAY FIXED EFFECTS (TWFE) MODEL ---
# We will use this as our main model to compare against.
# It includes Neighborhood FE (EntityEffects) and Day-of-Week FE (C(day_of_week))

print("--- 2. BASELINE TWFE MODEL ---")
base_formula = (
    'avg_trip_minutes ~ 1 + day_mean_temp + precipitation + wind_speed + holiday '
    '+ EntityEffects + C(day_of_week)'
)

# Load the model
base_model = PanelOLS.from_formula(base_formula, data=df)

# Fit the model
base_results = base_model.fit()
print(base_results)
print("\n")


# --- ROBUSTNESS TEST 1: NON-LINEAR RELATIONSHIP (Squared Term) ---
print("--- TEST 1: NON-LINEAR RELATIONSHIP (Squared Term) ---")
# Create the squared term
df['temp_squared'] = df['day_mean_temp']**2

nonlinear_formula = (
    'avg_trip_minutes ~ 1 + day_mean_temp + temp_squared + precipitation '
    '+ wind_speed + holiday + EntityEffects + C(day_of_week)'
)

# Run and print the new model
model_nl = PanelOLS.from_formula(nonlinear_formula, data=df)
results_nl = model_nl.fit()
print(results_nl)
# INTERPRETATION: Look at the p-value for 'temp_squared'. 
# If it's significant (e.g., < 0.05), it suggests a non-linear relationship.
# If day_mean_temp is positive and temp_squared is negative, it's an "inverted-U" shape.
print("\n")


# --- ROBUSTNESS TEST 2: CLUSTERED STANDARD ERRORS ---
print("--- TEST 2: CLUSTERED STANDARD ERRORS (at Neighborhood level) ---")
# We use the *same* baseline model, but change the `.fit()` command
# We cluster by 'neighborhood', which is our "entity".
clustered_results = base_model.fit(cov_type='clustered', cluster_entity=True)
print(clustered_results)
# INTERPRETATION: The 'Coef.' will be identical to the baseline model.
# Look at the 'Std. Err.' and 'P>|t|' columns. They will be different.
# This shows you if your results are still significant after accounting for 
# correlation within neighborhoods.
print("\n")


# --- ROBUSTNESS TEST 3: SUB-SAMPLE ANALYSIS (Weekends vs. Weekdays) ---
print("--- TEST 3: SUB-SAMPLE ANALYSIS ---")
# Create the sub-samples
df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday'])
df_weekdays = df[~df['is_weekend']]
df_weekends = df[df['is_weekend']]

# Run the *same* base formula on each sub-sample
model_wd = PanelOLS.from_formula(base_formula, data=df_weekdays)
results_wd = model_wd.fit()

model_we = PanelOLS.from_formula(base_formula, data=df_weekends)
results_we = model_we.fit()

print("--- WEEKDAY-ONLY RESULTS ---")
print(results_wd)
print("\n--- WEEKEND-ONLY RESULTS ---")
print(results_we)
# INTERPRETATION: Compare the 'Coef.' for 'day_mean_temp' in both models.
# Your hypothesis suggests the coefficient should be larger and/or more
# significant for the WEEKEND group.
print("\n")


# --- ROBUSTNESS TEST 4: SENSITIVITY TO CONTROLS ---
print("--- TEST 4: SENSITIVITY TO CONTROLS (Stepwise) ---")
# We run three models, adding controls at each step.

# Model 1: No controls
formula_1 = 'avg_trip_minutes ~ 1 + day_mean_temp + EntityEffects + C(day_of_week)'
res_1 = PanelOLS.from_formula(formula_1, data=df).fit()

# Model 2: Add weather controls
formula_2 = (
    'avg_trip_minutes ~ 1 + day_mean_temp + precipitation + wind_speed '
    '+ EntityEffects + C(day_of_week)'
)
res_2 = PanelOLS.from_formula(formula_2, data=df).fit()

# Model 3: Full model (same as baseline)
formula_3 = base_formula
res_3 = PanelOLS.from_formula(formula_3, data=df).fit()

print("--- MODEL 1 (No Controls) ---")
print(res_1.params[['day_mean_temp']])
print("\n--- MODEL 2 (Weather Controls) ---")
print(res_2.params[['day_mean_temp']])
print("\n--- MODEL 3 (Full Controls) ---")
print(res_3.params[['day_mean_temp']])
# INTERPRETATION: Look at the coefficient for 'day_mean_temp' in all three models.
# A robust finding means the coefficient's size and sign do not change drastically
# as you add more controls.
print("\n")


# --- ROBUSTNESS TEST 5: ALTERNATIVE VARIABLE DEFINITION ---
print("--- TEST 5: ALTERNATIVE VARIABLE (Binary 'warm_day') ---")
# Create your original binary variable
df['warm_day'] = (df['day_mean_temp'] > 65).astype(int)

# Create a new formula using 'warm_day' instead of 'day_mean_temp'
alt_formula = (
    'avg_trip_minutes ~ 1 + warm_day + precipitation + wind_speed + holiday '
    '+ EntityEffects + C(day_of_week)'
)

model_alt = PanelOLS.from_formula(alt_formula, data=df)
results_alt = model_alt.fit()
print(results_alt)
# INTERPRETATION: Look at the 'warm_day' coefficient. It should be
# positive and statistically significant, just as 'day_mean_temp' was.
# This shows your finding is not dependent on using a continuous variable.
print("\n")